In [1]:
import spacy as sp 
import torch 
import torch.nn as nn
import torch.functional as f 
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import time
import os
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Now we have a string from the other workbook we can tokenise it.   


In [2]:
text = open('stock_text.txt','r')

In [3]:
str = text.read()

In [4]:
#nlp = sp.load('en_core_web_sm')
#nlp.max_length = 8270369

In [5]:
#doc = nlp(str)

In [6]:
#words = [word.text for word in doc]
#print(words)

In [7]:
word_token = {}
token_word = {}

In [8]:
str_split = str.split(" ; ")

In [9]:
token = 0
for s in str_split:
    s = s.strip("\n")
    if s not in word_token.keys():
        word_token[s] = token
        token += 1 

In [10]:
for k in word_token.keys():
    t = word_token[k]
    token_word[t] = k 

In [11]:
#306996
input_list = []
for s in str_split:
    s = s.strip("\n")
    x = word_token[s]
    input_list.append(x)
len(input_list)

590241

In [12]:
# save token dict
f = open("word_token.pkl","wb")
pickle.dump(word_token,f)
f.close()
f = open("token_word.pkl","wb")
pickle.dump(token_word,f)
f.close()

In [13]:
class TransformerModel(nn.Module):
    def __init__(self,ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = "Transformer"
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        
        self.init_weights()
        
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz,sz))==1).transpose(0,1)
        mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask==1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [14]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [15]:
def data_process(raw_text_iter):
  data = torch.tensor(raw_text_iter)
  return data #torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [16]:
x = data_process(input_list)
val = int(len(input_list) * 0.80)
val 

472192

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [18]:
batch_size = 62
eval_batch_size = 62
train_data = batchify(x[0:val], batch_size)
val_data = batchify(x[val:], eval_batch_size)

In [19]:
train_data

tensor([[     0,   3017,      3,  ..., 211994, 217439, 222955],
        [     1,   3017,   7712,  ..., 211995, 217440, 222955],
        [     2,      3,   7713,  ..., 211996, 217441, 222956],
        ...,
        [  3014,   7709,  10514,  ..., 217436, 222952, 228373],
        [  3015,   7711,      3,  ..., 217437, 222953, 228374],
        [  3016,   7711,  10515,  ..., 217438, 222954,      9]])

In [20]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [21]:
ntokens = len(word_token) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [22]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [23]:
best_val_loss = float("inf")
epochs = 5 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    print("Starting the epoch")
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

Starting the epoch


/Users/jobymacbookpro/opt/anaconda3/envs/new_Strive/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/  217 batches | lr 5.00 | ms/batch 23429.49 | loss 10.26 | ppl 28469.18
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 5485.47s | valid loss 11.96 | valid ppl 157105.99
-----------------------------------------------------------------------------------------
Starting the epoch
| epoch   2 |   200/  217 batches | lr 4.51 | ms/batch 23421.55 | loss  8.83 | ppl  6845.19
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 5479.73s | valid loss 11.77 | valid ppl 129461.37
-----------------------------------------------------------------------------------------
Starting the epoch
| epoch   3 |   200/  217 batches | lr 4.29 | ms/batch 23264.01 | loss  8.73 | ppl  6205.02
-----------------------------------------------------------------------------------------
| end of epoch   3 | time: 5444.19s | valid loss 11.81 | valid ppl 133974.95
-----

In [24]:
torch.save(model, "mon_mod.pth")